In [3]:
using JuMP
using Gurobi
using CSV
using DataFrames
using CurricularAnalytics, JSON
using CurricularOptimization
using CA_API
using CurricularVisualization

include("../CurricularOptimization.jl/src/TransferArticulation.jl");
include("./Conversions.jl");

source_program = "Associate of Art"
destination_program = "Computing System"
# destination_program = "History"

university_courses = Dict{String,Any}();
university_catalogs = Dict{String,Any}();
kctcs_courses = JSON.parsefile("./bb_data/kctcs_courses.json");
kctcs_reqs = JSON.parsefile("./bb_data/Kentucky_college_programs/$(source_program).json");
eku_courses = JSON.parsefile("./bb_data/eku_courses.json");
eku_reqs = JSON.parsefile("./bb_data/Kentucky_university_programs/$(destination_program).json");

function add_courses_to_catalog(catalog, json_courses, destinationSchool)
    for course in json_courses
        c_split = split(course["code"], " ")
        prefix = c_split[1]
        num = c_split[2]
        if num === nothing
            continue
        end
        cid = mod(hash(course["name"] * prefix * num * destinationSchool), UInt32)
        cid = convert(Int, cid)
        course = Course(course["name"], course["maximum_credits"], prefix=prefix, num=num, institution=destinationSchool, id=cid)
        add_course!(catalog, course)
    end
end

source_school = "KCTCS" # name of the destination school
source_school_catalog = CourseCatalog(source_school, source_school);
add_courses_to_catalog(source_school_catalog, kctcs_courses, source_school);
source_requirements = parse_requirement_set(kctcs_reqs, source_school_catalog);

destination_school = "EKU"
destination_school_catalog = CourseCatalog(destination_school, destination_school);
add_courses_to_catalog(destination_school_catalog, eku_courses, destination_school);
destination_requirements_original = parse_requirement_set(eku_reqs, destination_school_catalog);

transfer_map_db = CSV.read("./bb_data/equivs.csv", DataFrame)
transfer_map_db = clean_transfer_file(
    transfer_map_db, source_school_catalog, destination_school_catalog
)
transfer_map, reverse_transfer_map = transfer_mapper(
    transfer_map_db, source_school_catalog, destination_school_catalog
)

course_id_array_source, reverse_course_id_map_source = get_course_ids(source_school_catalog)
course_id_array_destination, reverse_course_id_map_destination = get_course_ids(
    destination_school_catalog
)

time_hours = 0.01;
num_solutions = 1;

# requisite_destination = DataFrame()
requisite_source = DataFrame()

### run optimization with min_credit_sum objective function
course_to_requirement_source,
curriculum_source,
transferred_curriculum,
course_to_requirement_destination,
last_2_year_curriculum = 
transfer_articulation_min_credit_sum(source_requirements, destination_requirements_original, source_school_catalog, destination_school_catalog, 
transfer_map, reverse_transfer_map, course_id_array_source,
reverse_course_id_map_source, course_id_array_destination, reverse_course_id_map_destination;
requisite_source = requisite_source,
time_limit= 30, num_solutions=num_solutions)

# Create curriculum for source school
source_courses_list = [v for (k, v) in source_school_catalog.catalog];

curriculum_source = convert(Matrix{Int}, curriculum_source)
two_year_course_list = Array{Course,1}()
for (i, value) in enumerate(curriculum_source)
    if value > 0.5
        push!(two_year_course_list, source_courses_list[i])
    end
end

source_curriculum = CurricularAnalytics.Curriculum("Two Year", two_year_course_list)
source_degreeplan = CurricularOptimization.optimize_plan(source_curriculum, 4, 8, 25, [CurricularOptimization.balance_obj])
visualize(source_degreeplan)

# Create curriculum for destination school
destination_courses_list = [v for (k, v) in destination_school_catalog.catalog];

last_2_year_curriculum = convert(Matrix{Int}, last_2_year_curriculum)
four_year_course_list = Array{Course,1}()
for (i, value) in enumerate(last_2_year_curriculum)
    if value > 0.5
        push!(four_year_course_list, destination_courses_list[i])
    end
end

dest_curriculum = CurricularAnalytics.Curriculum("Four Year", four_year_course_list)
dest_degreeplan = CurricularOptimization.optimize_plan(dest_curriculum, 4, 8, 25, [CurricularOptimization.balance_obj])
visualize(dest_degreeplan)

thread = 1 warning: only found 77 / 78 columns around data row: 617. Filling remaining columns with `missing`
Set parameter Username
Academic license - for non-commercial use only - expires 2024-06-23
Set parameter TimeLimit to value 30
Set parameter PoolSolutions to value 1
Set parameter PoolSearchMode to value 2
Set parameter PoolGap to value 0
The root id is 18
The root id is 5
Set parameter PoolGap to value 0
Set parameter PoolSearchMode to value 2
Set parameter PoolSolutions to value 1
Set parameter TimeLimit to value 30
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-5700HQ CPU @ 2.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4718 rows, 142996 columns and 238596 nonzeros
Model fingerprint: 0x80888c9f
Model has 7725 general constraints
Variable types: 0 continuous, 142996 integer (142996 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e

Blink.AtomShell.Window(4, Blink.AtomShell.Electron(Process(`'C:\Users\Administrator\.julia\packages\Blink\mwJC9\deps\atom\electron.exe' 'C:\Users\Administrator\.julia\packages\Blink\mwJC9\src\AtomShell\main.js' port 8228`, ProcessRunning), Sockets.TCPSocket(Base.Libc.WindowsRawSocket(0x00000000000018f4) active, 0 bytes waiting), Dict{String, Any}("callback" => Blink.var"#1#2"())), Blink.Page(4, WebSocket(server, CONNECTED), Dict{String, Any}("webio" => Blink.AtomShell.var"#22#23"{Blink.AtomShell.WebIOBlinkComm}(Blink.AtomShell.WebIOBlinkComm(Blink.AtomShell.Window(#= circular reference @-5 =#))), "callback" => Blink.var"#1#2"()), Distributed.Future(1, 1, 4, ReentrantLock(nothing, Base.GenericCondition{Base.Threads.SpinLock}(Base.InvasiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), 0), Some(true))), Task (done) @0x0000000077ebdb30)